In [1]:
import ccxt
import numpy as np
import pandas as pd
import datetime as dt
from dateutil import tz
import matplotlib.pyplot as plt

import json
import time
import os
import sys

In [2]:
src_path = '../src/'
sys.path.append(os.path.abspath(src_path))
import func_get
import func_signal
import func_backtest

In [3]:
start_ts = time.time()

In [4]:
interval_dict = {
    '1m': 1,
    '5m': 5,
    '15m': 15,
    '30m': 30,
    '1h': 60,
    '2h': 120,
    '4h': 240,
    '1d': 1440
}

In [5]:
with open('../../../_keys/ftx/read.json') as keys_file:
    keys_dict = json.load(keys_file)
    
exchange = ccxt.ftx({'apiKey': keys_dict['apiKey'],
                     'secret': keys_dict['secret'],
                     'enableRateLimit': True})

### Config

In [6]:
project_name = 'cross'

In [7]:
start_date = dt.datetime(2022, 1, 1)
start_hour = 1
end_date = dt.datetime.today() - dt.timedelta(days=1)
reinvest_profit_flag = True

In [8]:
config_params = {
    'safety_ohlcv_range': 1000,
    'action_timeframe': '15m',
    'target_side': ['buy'],
    'base': {
        'symbol': ['BTC-PERP'],
        'open': {
            '15m': {
                'cross_sma': {
                    'check': ['check_signal_side_change'],
                    'look_back': 1,
                    'short_windows': 15,
                    'long_windows': 200,
                    'revert': False
                }
            }
        },
        'close': {
            '15m': {
                'cross_sma': {
                    'check': ['check_signal_side'],
                    'look_back': 1,
                    'short_windows': 15,
                    'long_windows': 200,
                    'revert': False
                }
            }
        }
    },
    'lead': {
        'symbol': [],
        'open': {
        },
        'close': {
        }
    },
    'tp': {
        'stop_percent': 100,
        'price_percent': None,
        'signal': None,
    },
    'sl': {
        'stop_percent': 100,
        'price_percent': None,
        'signal': None
    },
    'action_percent': 10,
    'leverage': 10,
    'taker_fee_percent': 0.07
}

### Get data

In [9]:
ohlcv_df_dict = func_get.get_data(exchange, start_date, end_date, start_hour, interval_dict, config_params)

base: timeframe 1/1 symbol 1/1 date 1/186
base: timeframe 1/1 symbol 1/1 date 2/186
base: timeframe 1/1 symbol 1/1 date 3/186
base: timeframe 1/1 symbol 1/1 date 4/186
base: timeframe 1/1 symbol 1/1 date 5/186
base: timeframe 1/1 symbol 1/1 date 6/186
base: timeframe 1/1 symbol 1/1 date 7/186
base: timeframe 1/1 symbol 1/1 date 8/186
base: timeframe 1/1 symbol 1/1 date 9/186
base: timeframe 1/1 symbol 1/1 date 10/186
base: timeframe 1/1 symbol 1/1 date 11/186
base: timeframe 1/1 symbol 1/1 date 12/186
base: timeframe 1/1 symbol 1/1 date 13/186
base: timeframe 1/1 symbol 1/1 date 14/186
base: timeframe 1/1 symbol 1/1 date 15/186
base: timeframe 1/1 symbol 1/1 date 16/186
base: timeframe 1/1 symbol 1/1 date 17/186
base: timeframe 1/1 symbol 1/1 date 18/186
base: timeframe 1/1 symbol 1/1 date 19/186
base: timeframe 1/1 symbol 1/1 date 20/186
base: timeframe 1/1 symbol 1/1 date 21/186
base: timeframe 1/1 symbol 1/1 date 22/186
base: timeframe 1/1 symbol 1/1 date 23/186
base: timeframe 1/1 

### Add signal

In [10]:
ohlcv_df_dict = func_signal.add_signal(start_date, ohlcv_df_dict, interval_dict, config_params)

base add cross_sma to BTC-PERP 15m
base add cross_sma to BTC-PERP 15m


### Back test

In [11]:
action_time_list = func_get.gen_action_time_list(config_params, ohlcv_df_dict)
max_open_timeframe = func_backtest.get_max_open_timeframe(config_params, interval_dict)

In [12]:
budget = 100

transaction_dict = {
    'symbol': [],
    'side': [],
    'amount': [],
    'open_time': [],
    'open_price': [],
    'close_time': [],
    'close_price': [],
    'value': [],
    'notional': [],
    'profit': [],
    'profit_percent': []
}

position_dict = {}

if reinvest_profit_flag == True:
    budget_dict = {
        'time': [],
        'budget': []
    }
    
max_drawdown = 0

In [13]:
if reinvest_profit_flag == True:
    min_budget = budget

time_count = 0

for signal_time in action_time_list:
    action_time = signal_time + dt.timedelta(minutes=interval_dict[config_params['action_timeframe']])
    
    time_count += 1
    print(f"Time {action_time}: progress {time_count}/{len(action_time_list)}")

    '''
    Close positions
    '''
    for symbol in [x for x in position_dict.keys()]:
        print(f"  {symbol}")
        budget, max_drawdown, position_dict, transaction_dict = func_backtest.close_position(symbol, signal_time, max_drawdown, config_params, budget, reinvest_profit_flag, ohlcv_df_dict, position_dict, transaction_dict, interval_dict)

    '''
    Open positions
    '''
    max_position = int(1 / (config_params['action_percent'] / 100))

    for symbol in [x for x in config_params['base']['symbol'] if x not in position_dict.keys()][:max_position]:
        print(f"  {symbol}")
        position_dict = func_backtest.open_position(symbol, signal_time, max_open_timeframe, config_params, budget, ohlcv_df_dict, position_dict, interval_dict)    
    
    '''
    Update budget
    '''
    if reinvest_profit_flag == True:
        budget_dict['time'].append(signal_time)
        budget_dict['budget'].append(budget)

        if budget < min_budget:
            min_budget = budget

    if budget <= 0:
        print(f"Out of money at {action_time}")
        break
        
'''
Clear final position
'''
opened_symbol = list(position_dict.keys())

for symbol in opened_symbol:
    print(f" Clear {symbol} position")
    
    side = position_dict[symbol]['side']
    ohlcv_df = ohlcv_df_dict['base'][config_params['action_timeframe']][symbol]
    current_ohlcv_df = ohlcv_df[ohlcv_df['time'] == signal_time].reset_index(drop=True)
    close_price = current_ohlcv_df.loc[0, 'close']
    close_percent = 100
    
    budget, position_dict, transaction_dict = func_backtest.update_close_position(symbol, side, close_price, close_percent, signal_time, config_params, budget, reinvest_profit_flag, position_dict, transaction_dict, interval_dict)

Time 2022-01-01 00:15:00: progress 1/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-01-01 00:30:00: progress 2/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-01-01 00:45:00: progress 3/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-01-01 01:00:00: progress 4/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-01-01 01:15:00: progress 5/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-01-01 01:30:00: progress 6/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-01-01 01:45:00: progress 7/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-01-01 02:00:00: progress

     base BTC-PERP check_signal_side_change cross_sma 15m: buy
     buy: 0.002048212261811804
     price: 47259.0
     tp: inf
     sl: 0
Time 2022-01-03 20:30:00: progress 274/16804
  BTC-PERP
       side: buy
       price: 47259.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-01-03 20:45:00: progress 275/16804
  BTC-PERP
       side: buy
       price: 47235.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-01-03 21:00:00: progress 276/16804
  BTC-PERP
       side: buy
       price: 47137.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-01-03 21:15:00: progress 277/16804
  BTC-PERP
       side: buy
       price: 46955.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-01-03 21:30:00: progress 278/16804
  BTC-PERP
  

     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-01-06 07:15:00: progress 509/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-01-06 07:30:00: progress 510/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-01-06 07:45:00: progress 511/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-01-06 08:00:00: progress 512/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-01-06 08:15:00: progress 513/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-01-06 08:30:00: progress 514/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-01-06 08:45:00: progress 515/16804
  BTC-PERP
     base BTC-PERP c

       side: buy
       price: 41812.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-01-09 12:45:00: progress 819/16804
  BTC-PERP
       side: buy
       price: 41821.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-01-09 13:00:00: progress 820/16804
  BTC-PERP
       side: buy
       price: 41723.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-01-09 13:15:00: progress 821/16804
  BTC-PERP
       side: buy
       price: 41701.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-01-09 13:30:00: progress 822/16804
  BTC-PERP
       side: buy
       price: 41735.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-01-09 13:45:00: progress 823/16804
  BTC-

     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-01-11 14:30:00: progress 1018/16804
  BTC-PERP
       side: buy
       price: 42161.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-01-11 14:45:00: progress 1019/16804
  BTC-PERP
       side: buy
       price: 42111.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-01-11 15:00:00: progress 1020/16804
  BTC-PERP
       side: buy
       price: 42095.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-01-11 15:15:00: progress 1021/16804
  BTC-PERP
       side: buy
       price: 42195.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-01-11 15:30:00: progress 1022/16804
  BTC-PERP
       side: buy
       price: 42221.0
       sl: 0
       sl_fl

...

     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-06-18 18:15:00: progress 16201/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-06-18 18:30:00: progress 16202/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-06-18 18:45:00: progress 16203/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-06-18 19:00:00: progress 16204/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-06-18 19:15:00: progress 16205/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-06-18 19:30:00: progress 16206/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-06-18 19:45:00: progress 16207/16804
  BTC-PERP
     b

     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-06-20 09:45:00: progress 16359/16804
  BTC-PERP
       side: buy
       price: 19647.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-06-20 10:00:00: progress 16360/16804
  BTC-PERP
       side: buy
       price: 19830.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-06-20 10:15:00: progress 16361/16804
  BTC-PERP
       side: buy
       price: 19827.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-06-20 10:30:00: progress 16362/16804
  BTC-PERP
       side: buy
       price: 19875.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-06-20 10:45:00: progress 16363/16804
  BTC-PERP
       side: buy
       price: 19991.0
       sl: 0
       

     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-06-21 16:45:00: progress 16483/16804
  BTC-PERP
       side: buy
       price: 21147.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-06-21 17:00:00: progress 16484/16804
  BTC-PERP
       side: buy
       price: 21145.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-06-21 17:15:00: progress 16485/16804
  BTC-PERP
       side: buy
       price: 21092.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-06-21 17:30:00: progress 16486/16804
  BTC-PERP
       side: buy
       price: 21038.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-06-21 17:45:00: progress 16487/16804
  BTC-PERP
       side: buy
       price: 21147.0
       sl: 0
       

     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-06-22 23:30:00: progress 16606/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-06-22 23:45:00: progress 16607/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-06-23 00:00:00: progress 16608/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-06-23 00:15:00: progress 16609/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-06-23 00:30:00: progress 16610/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-06-23 00:45:00: progress 16611/16804
  BTC-PERP
     base BTC-PERP check_signal_side_change cross_sma 15m: no_action
     No action
Time 2022-06-23 01:00:00: progress 16612/16804
  BTC-PERP
     b

     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-06-24 14:00:00: progress 16760/16804
  BTC-PERP
       side: buy
       price: 20765.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-06-24 14:15:00: progress 16761/16804
  BTC-PERP
       side: buy
       price: 20742.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-06-24 14:30:00: progress 16762/16804
  BTC-PERP
       side: buy
       price: 20720.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-06-24 14:45:00: progress 16763/16804
  BTC-PERP
       side: buy
       price: 20844.0
       sl: 0
       sl_flag: False
     base BTC-PERP check_signal_side cross_sma 15m: buy
     Not close
Time 2022-06-24 15:00:00: progress 16764/16804
  BTC-PERP
       side: buy
       price: 20836.0
       sl: 0
       

In [14]:
end_ts = time.time()
time_process = end_ts - start_ts
print(f"Time process: {round(time_process / 60, 2)} minutes")

Time process: 1.01 minutes


### Result

In [15]:
transaction_df = pd.DataFrame(transaction_dict)
transaction_df.to_csv(f'transactions/{project_name}.csv', index=False)
transaction_df

,symbol,side,amount,open_time,open_price,close_time,close_price,value,notional,profit,profit_percent
0,BTC-PERP,buy,0.002088,2022-01-02 00:45:00,47887.0,2022-01-02 13:00:00,47093.0,100.000000,100.000000,-1.796909,-1.795652
1,BTC-PERP,buy,0.002077,2022-01-02 17:30:00,47290.0,2022-01-03 04:15:00,47172.0,98.203091,98.203091,-0.382353,-0.389077
2,BTC-PERP,buy,0.002065,2022-01-03 06:45:00,47371.0,2022-01-03 09:15:00,46941.0,97.820737,97.820737,-1.024274,-1.046360
3,BTC-PERP,buy,0.002048,2022-01-03 20:15:00,47259.0,2022-01-03 22:15:00,46988.0,96.796463,96.796463,-0.690192,-0.712536
4,BTC-PERP,buy,0.002030,2022-01-04 22:00:00,47341.0,2022-01-05 02:00:00,45731.0,96.106271,96.106271,-3.400698,-3.536002
...,...,...,...,...,...,...,...,...,...,...,...
85,BTC-PERP,buy,0.002531,2022-06-16 05:15:00,22429.0,2022-06-16 16:15:00,21471.0,56.762794,56.762794,-2.502255,-4.405182
86,BTC-PERP,buy,0.002784,2022-06-19 20:30:00,19493.0,2022-06-19 22:00:00,19443.0,54.260539,54.260539,-0.215047,-0.396046
87,BTC-PERP,buy,0.002753,2022-06-19 23:15:00,19628.0,2022-06-22 09:15:00,20345.0,54.045492,54.045492,1.897206,3.507932
88,BTC-PERP,buy,0.002706,2022-06-23 18:00:00,20674.0,2022-06-23 23:00:00,20332.0,55.942698,55.942698,-1.003105,-1.791839


In [16]:
if reinvest_profit_flag == True:
    print(f"Final budget: {round(budget, 2)}")
else:
    print("Not reinvest profit")

Final budget: 54.58


In [17]:
if reinvest_profit_flag == True:
    return_percent = (budget_dict['budget'][-1] - budget_dict['budget'][0]) / budget_dict['budget'][0] * 100
    print(f"Return: {round(return_percent, 2)}%")
else:
    print("Not reinvest profit")

Return: -45.06%


In [18]:
if reinvest_profit_flag == True:
    fig, ax = plt.subplots(figsize=(30, 8))
    ax.plot(budget_dict['time'], budget_dict['budget'])
else:
    print("Not reinvest profit")

In [19]:
if reinvest_profit_flag == True:
    print(f"Max drawdown: {round(max_drawdown * 100, 2)}%")

Max drawdown: 8.18%


In [20]:
win_rate = len(transaction_df[transaction_df['profit'] > 0]) / len(transaction_df)
print(f"Win rate: {round(win_rate * 100, 2)}%")

Win rate: 15.56%


In [21]:
for symbol in transaction_df['symbol'].unique():
    symbol_df = transaction_df[transaction_df['symbol'] == symbol]
    win_rate = len(symbol_df[symbol_df['profit'] > 0]) / len(symbol_df)
    print(f"{symbol} win rate: {round(win_rate * 100, 2)}%")

BTC-PERP win rate: 15.56%


In [22]:
print(f"Distribution")
print(pd.DataFrame(transaction_df['profit_percent'].describe()))

Distribution
       profit_percent
count       90.000000
mean        -0.633881
std          2.751612
min         -7.414479
25%         -1.702280
50%         -0.816600
75%         -0.412175
max         14.542984


In [23]:
for symbol in transaction_df['symbol'].unique():
    temp_return = transaction_df[transaction_df['symbol'] == symbol]['profit_percent']
    
    print(f"Distribution {symbol}:")
    print(pd.DataFrame(temp_return.describe()))
    print("\n")

Distribution BTC-PERP:
       profit_percent
count       90.000000
mean        -0.633881
std          2.751612
min         -7.414479
25%         -1.702280
50%         -0.816600
75%         -0.412175
max         14.542984




In [24]:
fig, ax = plt.subplots(figsize=(30, 8))
_ = ax.hist(transaction_df['profit_percent'], bins=10)